### LSTM for MNIST

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets.mnist import load_data

def to_one_hot(np_arr, num_classes):
    a = np.zeros((len(np_arr), num_classes))
    a[np.arange(len(np_arr)), np_arr] = 1
    return a

batch_size = 128

(x_train, y_train), (x_test, y_test) = load_data('/tmp/mnist.npz')    
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_one_hot(y_train, 10)
y_test = to_one_hot(y_test, 10)

x_train = x_train[:5000]
y_train = y_train[:5000]

def get_batches():
    instance_indices = list(range(len(x_train)))
    np.random.shuffle(instance_indices)
    
    for start in range(0, len(instance_indices), batch_size):
        batch = instance_indices[start:start+batch_size]
        x = np.array(x_train)[batch]
        y = np.array(y_train)[batch]
        yield x, y

### Explicit fully connected layer

In [2]:
lstm_size = 128
num_classes = 10
epochs = 10

with tf.name_scope('inputs'):
    x = tf.placeholder(tf.float32, shape=(None, 28, 28), name='x')
    y = tf.placeholder(tf.int32, shape=(None, 10), name='y')

with tf.name_scope('rnn'):
    cells = [tf.nn.rnn_cell.LSTMCell(lstm_size) for _ in range(2)]
    lstm_cell = tf.nn.rnn_cell.MultiRNNCell(cells)
    
    outputs, (cell_state, states) = tf.nn.dynamic_rnn(
        lstm_cell, x, 
        dtype=tf.float32
    )
    state = states[-1]
        
with tf.name_scope('loss'):
    # Fully connected layer. 
    wy = tf.Variable(
        tf.truncated_normal(
            [lstm_size, num_classes], mean=0.0,
            stddev=.01, dtype=tf.float32
        ),
        name='wy'
    )
    bias = tf.Variable(
        tf.truncated_normal(
            [num_classes], mean=0.0,
            stddev=.01, dtype=tf.float32
        ),
        name='bias'
    )
    logits = tf.matmul(state, wy) + bias
    
    logits = tf.layers.dense(state, tf.float32)
    print(logits)
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=y,
        logits=logits,
        name='loss'
    ))
    correct = tf.equal(
        tf.argmax(y, 1), 
        tf.argmax(logits, 1)
    )
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    
    train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

with tf.Session() as sess:
    sess.run(tf.initializers.global_variables())
    
    for epoch in range(10):
        loss, acc = sess.run([cross_entropy, accuracy], feed_dict={
            x: x_train, y: y_train
        })
        print('Loss, Accuracy at epoch %d: %f %f' % (epoch, loss, acc))
                      
        for x_batch, y_batch in get_batches():                    
            _ = sess.run([train_step], feed_dict={
                x: x_batch, y: y_batch
            })
    loss, acc = sess.run([cross_entropy, accuracy], feed_dict={
        x: x_test, y: y_test
    })
    print('Loss, Accuracy at test: %f %f' % (loss, acc))

Loss, Accuracy at epoch 0: 2.302449 0.102200
Loss, Accuracy at epoch 1: 1.424858 0.533600
Loss, Accuracy at epoch 2: 0.888599 0.721400
Loss, Accuracy at epoch 3: 0.607078 0.801800
Loss, Accuracy at epoch 4: 0.432144 0.872800
Loss, Accuracy at epoch 5: 0.445102 0.852600
Loss, Accuracy at epoch 6: 0.303863 0.907000
Loss, Accuracy at epoch 7: 0.232513 0.937400
Loss, Accuracy at epoch 8: 0.172309 0.955000
Loss, Accuracy at epoch 9: 0.165635 0.953600
Loss, Accuracy at test: 0.272021 0.918900


### Minimal

In [3]:
lstm_size = 128
num_classes = 10
epochs = 10

with tf.name_scope('inputs'):
    x = tf.placeholder(tf.float32, shape=(None, 28, 28), name='x')
    y = tf.placeholder(tf.int32, shape=(None, 10), name='y')

with tf.name_scope('rnn'):
    cells = [tf.nn.rnn_cell.LSTMCell(lstm_size) for _ in range(2)]
    lstm_cell = tf.nn.rnn_cell.MultiRNNCell(cells)
    
    _, (_, states) = tf.nn.dynamic_rnn(
        lstm_cell, x, 
        dtype=tf.float32
    )
    state = states[-1]
        
with tf.name_scope('loss'):
    logits = tf.layers.dense(state, 10)
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=y,
        logits=logits,
        name='loss'
    ))
    correct = tf.equal(
        tf.argmax(y, 1), 
        tf.argmax(logits, 1)
    )
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    
    train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

with tf.Session() as sess:
    sess.run(tf.initializers.global_variables())
    
    for epoch in range(10):
        loss, acc = sess.run([cross_entropy, accuracy], feed_dict={
            x: x_train, y: y_train
        })
        print('Loss, Accuracy at epoch %d: %f %f' % (epoch, loss, acc))
                      
        for x_batch, y_batch in get_batches():                    
            _ = sess.run([train_step], feed_dict={
                x: x_batch, y: y_batch
            })
    loss, acc = sess.run([cross_entropy, accuracy], feed_dict={
        x: x_test, y: y_test
    })
    print('Loss, Accuracy at test: %f %f' % (loss, acc))

Tensor("loss/dense/BiasAdd:0", shape=(?, 10), dtype=float32)
Loss, Accuracy at epoch 0: 2.297145 0.074600
Loss, Accuracy at epoch 1: 1.137755 0.599200
Loss, Accuracy at epoch 2: 0.609937 0.802000
Loss, Accuracy at epoch 3: 0.387284 0.882400
Loss, Accuracy at epoch 4: 0.312382 0.899600
Loss, Accuracy at epoch 5: 0.228568 0.927600
Loss, Accuracy at epoch 6: 0.174147 0.946400
Loss, Accuracy at epoch 7: 0.143565 0.959200
Loss, Accuracy at epoch 8: 0.134101 0.958400
Loss, Accuracy at epoch 9: 0.136323 0.956600
Loss, Accuracy at test: 0.242499 0.926200


### Odd-Even Sequences Example

In [1]:
import numpy as np
import tensorflow as tf

digit_to_word_map = {
    1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five',
    6: 'Six', 7: 'Seven', 8: 'Eight', 9: 'Nine'
}

digit_to_word_map[0] = 'PAD'

even_sentences = []
odd_sentences = []
seqlens = []
for i in range(10000):
    rand_seq_len = np.random.choice(range(3,7))
    seqlens.append(rand_seq_len)
    rand_odd_ints = np.random.choice(range(1,10,2), rand_seq_len)
    rand_even_ints = np.random.choice(range(2,10,2), rand_seq_len)    
    
    if rand_seq_len < 6:
        rand_odd_ints = np.append(rand_odd_ints, [0]*(6-rand_seq_len))
        rand_even_ints = np.append(rand_even_ints, [0]*(6-rand_seq_len))        
        
    even_sentences.append(' '.join([digit_to_word_map[r] for r in rand_even_ints]))
    odd_sentences.append(' '.join([digit_to_word_map[r] for r in rand_odd_ints]))    
    
data = even_sentences + odd_sentences
seqlens *= 2
    
print(even_sentences[0:6])
print(odd_sentences[0:6])
print(seqlens[0:6])

word2index_map = {}
index = 0
for sent in data:
    for word in sent.lower().split():
        if word not in word2index_map:
            word2index_map[word] = index
            index += 1

index2word_map = {index: word for word, index in word2index_map.items()}
vocabulary_size = len(index2word_map)

labels = [1]*10000 + [0]*10000
for i in range(len(labels)):
    label = labels[i]
    one_hot_encoding = [0]*2
    one_hot_encoding[label] = 1
    labels[i] = one_hot_encoding
    
data_indices = list(range(len(data)))
np.random.shuffle(data_indices)
data = np.array(data)[data_indices]

labels = np.array(labels)[data_indices]
seqlens = np.array(seqlens)[data_indices]

train_x = data[:10000]
train_y = labels[:10000]
train_seqlens = seqlens[:10000]

test_x = data[10000:]
test_y = labels[10000:]
test_seqlens = seqlens[10000:]
        
def get_sentence_batch(batch_size, data_x, data_y, data_seqlens):
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    x = [[word2index_map[word] for word in data_x[i].lower().split()] for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x, y, seqlens

['Six Eight Six PAD PAD PAD', 'Two Two Eight Four Four Six', 'Six Four Eight PAD PAD PAD', 'Four Eight Six Two Two PAD', 'Eight Six Eight Eight Six Eight', 'Four Four Six Two Six Eight']
['One Three Nine PAD PAD PAD', 'One One Three One One Seven', 'Nine Three Five PAD PAD PAD', 'One Nine Five Three Nine PAD', 'Nine Seven Seven Nine One Three', 'One Nine One Five One One']
[3, 6, 3, 5, 6, 6]


In [2]:
import tensorflow as tf 
import numpy as np

lstm_size = 128
num_classes = 2
epochs = 10

with tf.name_scope('inputs'):
    x = tf.placeholder(tf.int32, shape=(None, 6), name='x')
    y = tf.placeholder(tf.int32, shape=(None, 2), name='y')
    seqlens = tf.placeholder(tf.int32, shape=(None,), name='seqlens')
    
    emb_size = 10
    emb_matrix = tf.constant(np.random.random((len(word2index_map), emb_size)), dtype=tf.float32)
    emb = tf.nn.embedding_lookup(
        emb_matrix, x
    )

with tf.name_scope('rnn'):
    cells = [tf.nn.rnn_cell.LSTMCell(lstm_size) for _ in range(2)]
    lstm_cell = tf.nn.rnn_cell.MultiRNNCell(cells)
    
    _, (_, states) = tf.nn.dynamic_rnn(
        lstm_cell, emb, 
        dtype=tf.float32,
        sequence_length=seqlens
    )
    state = states[-1]
        
with tf.name_scope('loss'):
    logits = tf.layers.dense(state, 2)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=y,
        logits=logits,
        name='loss'
    ))
    predictions = tf.argmax(logits, 1)
    correct = tf.equal(
        tf.argmax(y, 1), 
        predictions
    )
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    
    train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)
    acc_metric, update_op = tf.metrics.accuracy(tf.argmax(y, 1), predictions)  
    print(acc_metric)

with tf.Session() as sess:
    sess.run(tf.initializers.global_variables())
    sess.run(tf.local_variables_initializer())
    
    for step in range(1000):        
        batch_x, batch_y, batch_seqlens = get_sentence_batch(100, train_x, train_y, train_seqlens)

        _ = sess.run(train_step, feed_dict={
            x: batch_x, y: batch_y, seqlens: batch_seqlens
        })
        
        if step % 100 == 0:
            loss, acc = sess.run([cross_entropy, accuracy], feed_dict={
                x: batch_x, y: batch_y, seqlens: batch_seqlens
            })
            print('Loss, Accuracy at epoch %d: %f %f' % (step, loss, acc))
        
    test_x, test_y, test_seqlens = get_sentence_batch(100000, test_x, test_y, test_seqlens)
    loss, acc, update_op_eval = sess.run([cross_entropy, accuracy, update_op], feed_dict={
        x: test_x, y: test_y, seqlens: test_seqlens
    })
    print('Loss, Accuracy at test: %f %f' % (loss, acc))
    print('Mean after batch: %f' % (update_op_eval))

Tensor("loss/accuracy/value:0", shape=(), dtype=float32)
Loss, Accuracy at epoch 0: 0.707505 0.410000
Loss, Accuracy at epoch 100: 0.432970 1.000000
Loss, Accuracy at epoch 200: 0.054936 1.000000
Loss, Accuracy at epoch 300: 0.025139 1.000000
Loss, Accuracy at epoch 400: 0.005556 1.000000
Loss, Accuracy at epoch 500: 0.002443 1.000000
Loss, Accuracy at epoch 600: 0.001827 1.000000
Loss, Accuracy at epoch 700: 0.001019 1.000000
Loss, Accuracy at epoch 800: 0.000413 1.000000
Loss, Accuracy at epoch 900: 0.000387 1.000000
Loss, Accuracy at test: 0.000296 1.000000
Mean after batch: 1.000000


### LSTM estimator

In [7]:
import tensorflow as tf 
import numpy as np

lstm_size = 128
num_classes = 2
epochs = 10

def model_fn(features, labels, mode):
    x, seqlens = features['x'], features['seqlens']
    y = np.array(labels)
    
    # Embeddings.
    emb_size = 10
    emb_matrix = tf.constant(np.random.random((len(word2index_map), emb_size)), dtype=tf.float32)
    emb = tf.nn.embedding_lookup(
        emb_matrix, x
    )

    # RNN.
    cells = [tf.nn.rnn_cell.LSTMCell(lstm_size) for _ in range(2)]
    lstm_cell = tf.nn.rnn_cell.MultiRNNCell(cells)

    _, (_, states) = tf.nn.dynamic_rnn(
        lstm_cell, emb, 
        dtype=tf.float32,
        sequence_length=seqlens
    )
    state = states[-1]

    # Loss.
    logits = tf.layers.dense(state, 2)
    predictions = tf.argmax(logits, 1)
         
    if mode == tf.estimator.ModeKeys.PREDICT:    
        predict_output = { 'values': predictions }
        return tf.estimator.EstimatorSpec(
          mode=mode,
          predictions=predict_output
        )
    else:
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=y,            
            logits=logits,
            name='loss'
        ))
        correct = tf.equal(
            tf.argmax(y, 1), 
            predictions
        )
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    
        train_step = tf.train.AdamOptimizer(0.0001).minimize(
            cross_entropy,
            global_step=tf.train.get_or_create_global_step()
        )

        metrics = {
            'accuracy': tf.metrics.accuracy(tf.argmax(y, 1), predictions)
        }
        
        if mode == tf.estimator.ModeKeys.EVAL:
            return tf.estimator.EstimatorSpec(
              mode=mode,
              loss=cross_entropy,            
              eval_metric_ops=metrics
            )
        elif mode == tf.estimator.ModeKeys.TRAIN:
            return tf.estimator.EstimatorSpec(
              mode=mode,
              loss=cross_entropy,
              train_op=train_step
            )    
    
def input_fn(x, y, seqlens, batch_size=1000000):
    x, y, seqlens = get_sentence_batch(batch_size, x, y, seqlens)
    features = {
        'x': x,
        'seqlens': seqlens
    }
    return features, y
    
estimator = tf.estimator.Estimator(
    model_fn,
    model_dir="/code/checkpoints"
)

estimator.train(lambda: input_fn(train_x, train_y, train_seqlens, batch_size=100), steps=300)
# metrics = estimator.evaluate(lambda: input_fn(train_x, train_y, train_seqlens), steps=1)

predictions = estimator.predict(lambda: input_fn(train_x, train_y, train_seqlens))

for p in predictions:
    print(p)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/code/checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6847c35cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph w

{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'valu

{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'valu

{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'valu

{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'valu

...

{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'valu

{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'valu

{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'valu

{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 0}
{'valu

KeyboardInterrupt: 

In [12]:
!rm -r /code/checkpoints/*

In [6]:
import itertools

predictions = list(itertools.islice(estimator.predict(lambda: input_fn(train_x, train_y, train_seqlens)),100))

for p in predictions:
    print(p)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /code/checkpoints/model.ckpt-2600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 0}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 0}
{'values': 1}
{'values': 1}
{'values': 1}
{'values': 1}
{'v